In [ ]:
# simulate_auv.py
import json
import numpy as np
from clustering import cluster_split, choose_cluster_head
from select_path_greedy import compute_travel_time, path_selection

def simulate_auv_cycle(data, r_sen=100, R=20, E_tx=5, E_rx=2, v_AUV=10, death_threshold=0.9):
    
    positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
    energies = {d["id"]: d["energy_node"] for d in data}
    node_ids = [d["id"] for d in data]
    total_nodes = len(node_ids)
    cycle = 0
    total_travel_time = 0.0

    while True:
        alive_ids = [nid for nid, e in energies.items() if e > 0] # chỉ xét các node còn sống
        alive_count = len(alive_ids)
        dead_count = total_nodes - alive_count
        dead_ratio = dead_count / total_nodes

        # điều kiện dừng
        if dead_ratio >= death_threshold:
            print(f"\nMạng sập: {dead_count}/{total_nodes} nodes chết ({dead_ratio*100:.1f}%)")
            break

        cycle += 1
        print(f"\nChu kỳ {cycle}: {alive_count}/{total_nodes} nodes hoạt động")

        # lấy pos của nodes còn sống
        alive_positions = np.array([positions[nid] for nid in alive_ids])

        # phân cụm các node còn sống (trả về danh sách cụm)
        clusters_raw = cluster_split(alive_positions, alive_ids, r_sen=r_sen, R=R)
        cluster_heads = []
        positions_dict = {}

        for c in clusters_raw:
            ch = choose_cluster_head(c, energies)
            cluster_heads.append(ch)
            # dùng tâm cụm để tính khoảng cách 
            positions_dict[ch] = np.mean(c["nodes"], axis=0)

        # nếu không có cluster head (quá ít node), dừng
        if len(cluster_heads) == 0:
            print("Không còn cluster head nào — dừng mô phỏng.")
            break

        # tính ma trận thời gian di chuyển và tìm đường
        travel_time = compute_travel_time(cluster_heads, positions_dict, v_AUV)
        best_path, best_time = path_selection(cluster_heads, travel_time, positions, v_AUV)

        # tạo danh sách chi tiết các segment
        O = np.array([0, 0, 0])
        segment_times = []

        # Từ O đến nút đầu tiên
        first_ch = cluster_heads[best_path[0]]
        start_dist = np.linalg.norm(positions_dict[first_ch] - O)
        start_time = start_dist / v_AUV
        segment_times.append((f"O → {first_ch}", start_time))

        # giữa các CH
        for i in range(1, len(best_path)):
            from_idx = cluster_heads[best_path[i-1]]
            to_idx = cluster_heads[best_path[i]]
            seg_time = travel_time[best_path[i-1]][best_path[i]]
            segment_times.append((f"{from_idx} → {to_idx}", seg_time))

        # nút cuối về O
        last_ch = cluster_heads[best_path[-1]]
        end_dist = np.linalg.norm(positions_dict[last_ch] - O)
        end_time = end_dist / v_AUV
        segment_times.append((f"{last_ch} → O", end_time))

        # tổng thời gian 
        cycle_time = best_time + start_time + end_time
        total_travel_time += cycle_time
        

    
        print("\nĐường đi tối ưu chi tiết (theo thứ tự thăm của AUV):")
        path_ids = [cluster_heads[i] for i in best_path]
        for idx, node_id in enumerate(path_ids, start=1):
            pos = positions_dict[node_id]
            print(f"  {idx:02d}. Node ID: {node_id:<3}  | Tọa độ: ({pos[0]:.2f}, {pos[1]:.2f}, {pos[2]:.2f})")
        print("về O(0, 0, 0)")
        print("\nThời gian từng chặng trong chu kỳ:")
        for idx, (seg_desc, t) in enumerate(segment_times, start=1):
            print(f"  {idx:02d}. {seg_desc:<20} → {t:.2f} s")
        print(f"Tổng thời gian chu kỳ {cycle}: {cycle_time:.2f} s")

        # cập nhật năng lượng cho các node
        for c in clusters_raw:
            ch = choose_cluster_head(c, energies)
            energies[ch] -= E_tx
            if energies[ch] < 0:
                energies[ch] = 0
            for nid in c["node_ids"]:
                if nid != ch:
                    energies[nid] -= E_rx
                    if energies[nid] < 0:
                        energies[nid] = 0

        # in trạng thái tóm tắt
        alive_count_after = len([e for e in energies.values() if e > 0])
        dead_count_after = total_nodes - alive_count_after
        print(f"Sau cập nhật năng lượng: {alive_count_after} nodes sống, {dead_count_after} nodes chết")

        # kiểm tra điều kiện dừng
        if (total_nodes - alive_count_after) / total_nodes >= death_threshold:
            print(f"\nMạng sập sau cập nhật: {(total_nodes - alive_count_after)}/{total_nodes} nodes chết")
            break

    print(f"\nTổng số chu kỳ hoàn thành: {cycle}")
    print(f"Tổng thời gian di chuyển của AUV: {total_travel_time:.2f} s")
    return cycle, total_travel_time


# chạy mô phỏng 
if __name__ == "__main__":
    with open("input_data/nodes_100.json", "r") as f:
        data = json.load(f)
    simulate_auv_cycle(data)



Chu kỳ 1: 100/100 nodes hoạt động


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subp


Đường đi tối ưu chi tiết (theo thứ tự thăm của AUV):
  01. Node ID: 18   | Tọa độ: (15.09, 23.93, 70.10)
  02. Node ID: 51   | Tọa độ: (66.49, 25.48, 77.55)
  03. Node ID: 58   | Tọa độ: (72.84, 19.97, 35.58)
  04. Node ID: 37   | Tọa độ: (86.15, 72.41, 25.89)
  05. Node ID: 84   | Tọa độ: (69.75, 71.21, 82.77)
  06. Node ID: 10   | Tọa độ: (19.96, 81.90, 50.80)
  07. Node ID: 74   | Tọa độ: (27.19, 75.45, 16.50)
  08. Node ID: 63   | Tọa độ: (26.60, 28.70, 22.39)
về O(0, 0, 0)

Thời gian từng chặng trong chu kỳ:
  01. O → 18               → 7.56 s
  02. 18 → 51              → 5.20 s
  03. 51 → 58              → 4.28 s
  04. 58 → 37              → 5.50 s
  05. 37 → 84              → 5.92 s
  06. 84 → 10              → 6.01 s
  07. 10 → 74              → 3.56 s
  08. 74 → 63              → 4.71 s
  09. 63 → O               → 4.51 s
Tổng thời gian chu kỳ 1: 59.77 s
Sau cập nhật năng lượng: 100 nodes sống, 0 nodes chết

Chu kỳ 2: 100/100 nodes hoạt động

Đường đi tối ưu chi tiết (theo th